In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [ ]:
import numpy as np


from ICARUS.Core.Units import calc_mach, calc_reynolds
from ICARUS.Database.db import DB
from ICARUS.Airfoils.airfoilD import AirfoilD
from ICARUS.Workers.solver import Solver
from ICARUS.Software.OpenFoam.filesOpenFoam import MeshType

# Reynolds And Mach and AoA

In [ ]:
db= DB()

In [ ]:
# Assumptions
chord_max: float = 0.18
chord_min: float = 0.11
u_max: float = 30.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_min: float = calc_mach(10, speed_of_sound)
mach_max: float = calc_mach(30, speed_of_sound)
mach = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds = np.logspace(
    start=np.log10(reynolds_min),
    stop=np.log10(reynolds_max),
    num=2,
    base=10,
)
REYN = reynolds_max

# ANGLE OF ATTACK SETUP
aoa_max: float = 2
aoa_min: float = -2
num_of_angles: float = (aoa_max - aoa_min) * 2 + 1
angles = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)
airfoil_name: str = "2418"
airfoil: AirfoilD = AirfoilD.naca(naca=airfoil_name, n_points=200)

ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}

In [ ]:
print(f"Nominal: Mach {MACH}\tReynolds {REYN}")


In [ ]:
cleaning = False
calcF2W = False
calcOpenFoam = True
calcXFoil = False

# Get Airfoil

In [ ]:
airf: AirfoilD = AirfoilD.naca('0012',n_points=200)
airf.plot()

# Foil2Wake

In [ ]:
# if cleaning :
    # airf.clean_results(f2w.remove_results, [airf.REYNDIR, airf.HOMEDIR,angles])
if calcF2W :
    # Import Solver
    from ICARUS.Software.F2Wsection.f2w_section import get_f2w_section
    f2w_s = get_f2w_section(db)

    # Import Analysis
    analysis: str = f2w_s.available_analyses_names()[0]  # Run
    f2w_s.set_analyses(analysis)

    # Get Options
    options = f2w_s.get_analysis_options(verbose=True)
    solver_parameters = f2w_s.get_solver_parameters()

    # Set Options
    options.db.value = db
    options.airfoil.value = airfoil
    options.reynolds.value = reynolds
    options.mach.value = MACH
    options.f_trip_low.value = ftrip_low
    options.f_trip_upper.value = ftrip_up
    options.all_angles.value = angles
    f2w_s.print_analysis_options()

    # RUN
    f2w_s.run()

    # Get Results
    polars = f2w_s.get_results()


# Xfoil

In [ ]:
if calcXFoil:
    airfoils = [airf.name]#["0008"]
    from ICARUS.Software.Xfoil.xfoil import get_xfoil

    for airfoil in airfoils:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: AirfoilD = AirfoilD.naca(airfoil, n_points=200)
        # airf.plotAirfoil()
        xfoil: Solver = get_xfoil(db)

        # Import Analysis
        analysis: str = xfoil.available_analyses_names()[0]  # Run
        xfoil.set_analyses(analysis)

        # Get Options
        options = xfoil.get_analysis_options(verbose=True)
        solver_parameters = xfoil.get_solver_parameters()

        # Set Options
        options.db.value = db
        options.airfoil.value = airf
        options.reynolds.value = reynolds
        options.mach.value = MACH
        options.min_aoa.value = -5
        options.max_aoa.value = 5
        options.aoa_step.value = 1
        xfoil.print_analysis_options()

        # Set Solver Options
        solver_parameters.max_iter.value = 400
        solver_parameters.Ncrit.value = 1e-3
        solver_parameters.xtr.value = (0.1, 0.1)
        solver_parameters.print.value = False
        # xfoil.print_solver_options()
        
        # RUN
        xfoil.run()


# OpenFoam

In [30]:
if calcOpenFoam:
    airfoils: list[str] = [airf.name]#["0008"]
    from ICARUS.Software.OpenFoam.open_foam import get_open_foam

    for airfoil in airfoils:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: AirfoilD = AirfoilD.naca(airfoil, n_points=200)
        # airf.plotAirfoil()
        open_foam: Solver = get_open_foam(db)

        # Import Analysis
        analysis: str = open_foam.available_analyses_names()[0]  # Run
        open_foam.set_analyses(analysis)

        # Get Options
        options = open_foam.get_analysis_options(verbose=True)
        solver_parameters = open_foam.get_solver_parameters()

        # Set Options
        options.db.value = db
        options.airfoil.value = airf
        options.angles.value = angles
        options.reynolds.value = REYN
        options.mach.value = MACH
        open_foam.print_analysis_options()

        # Set Solver Options
        solver_parameters.mesh_type.value = MeshType.structAirfoilMesher
        solver_parameters.max_iterations.value = 20
        solver_parameters.silent.value = False
        # xfoil.print_solver_options()
        
        # RUN
        open_foam.run()

[autoreload of ICARUS.Software.OpenFoam.filesOpenFoam failed: Traceback (most recent call last):
  File "/home/tryfonas/Applications/anaconda3/envs/aero/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/tryfonas/Applications/anaconda3/envs/aero/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/home/tryfonas/Applications/anaconda3/envs/aero/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/tryfonas/data/Uni/Sof


Running airfoil 0012



Running Solver open_foam:
	Analysis SIMPLE_WITH_STRUCT_MESH...
Making Mesh for naca0012
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/m2.0000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/m1.5000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/m1.0000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/m0.5000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/0.00000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/0.50000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/1.00000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/1.50000 Ready to Run
/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/2.00000 Ready to Run
-2.0 deg: Simulation St



--> FOAM FATAL ERROR: (openfoam-2306)
cannot find file "/home/tryfonas/data/Uni/Software/hermes/Data/2D/NACA0012/Reynolds_3.462e05/m2.0000/511/p"

    From virtual Foam::autoPtr<Foam::ISstream> Foam::fileOperations::uncollatedFileOperation::readStream(Foam::regIOobject&, const Foam::fileName&, const Foam::word&, bool) const
    in file global/fileOperations/uncollatedFileOperation/uncollatedFileOperation.C at line 627.

FOAM exiting



-1.5 deg: Simulation Starting
-1.0 deg: Simulation Starting
-0.5 deg: Simulation Starting
0.0 deg: Simulation Starting
0.5 deg: Simulation Starting
1.0 deg: Simulation Starting
1.5 deg: Simulation Starting
2.0 deg: Simulation Starting
Analysis Completed
